#### Calculate the Congestion KPIs

In [184]:
import os
import itertools

import numpy as np
import pandas as pd
from simpledbf import Dbf5

In [185]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [186]:
from scipy.stats import trim_mean

In [187]:
S01_Existing = 'S01_Existing'
S03_HGAC = 'S03_HGAC'
S04_HG = 'S04_HG'
S06_HG_MT = 'S06_HG_MT'
S08_HG_HT_Sprawl_TDM_N3 = 'S08_HG_HT_Sprawl_TDM_N3'
S12_HG_HT_Infill_Pricing_N3 = 'S12_HG_HT_Infill_Pricing_N3'
S14_HG_HT_Infill_Transit_N3 = 'S14_HG_HT_Infill_Transit_N3'

In [219]:
scenario = 'Scenario_2017'
workspace = r'L:\DCS\Projects\_Legacy\60563434_SWIFT\400_Technical\SWIFT_Workspace\Scenarios'
grid_file = r'STM\STM_A\04_KPI_PreProcessor\02_Results\Grids_KPI.dbf'
grid_file = os.path.join(workspace, scenario, grid_file)
print("Input Grid File:", grid_file)

Input Grid File: L:\DCS\Projects\_Legacy\60563434_SWIFT\400_Technical\SWIFT_Workspace\Scenarios\Scenario_2017\STM\STM_A\04_KPI_PreProcessor\02_Results\Grids_KPI.dbf


In [189]:
df_grids = Dbf5(grid_file).to_dataframe()

In [190]:
# df_grids.info()

In [191]:
df_grids.fillna(0, inplace=True)

#### Travel Time Index

In [192]:
region_tti = np.sum(df_grids.DAILY_VHT) / np.sum(df_grids.DAILY_VHT.values - df_grids.DAILY_VHD.values)
print('Daily Study Area Unweighted Travel Time Index = {:.2f}'.format(float(region_tti)))

Daily Study Area Unweighted Travel Time Index = 1.84


In [193]:
def tti_weighted_by_mt(vmt, vht, vhd, v):
    weights = vmt / (v+1)
    tti = np.dot(vht, weights) / (np.dot(vht, weights) - np.dot(vhd, weights))
    return tti

In [194]:
region_tti_weighted_by_mt = tti_weighted_by_mt(df_grids.DAILY_VMT.values, df_grids.DAILY_VHT.values,df_grids.DAILY_VHD.values,df_grids.DAILY_VOL.values)
print('Daily Study Area Unweighted Travel Time Index = {:.2f}'.format(float(region_tti)))

Daily Study Area Unweighted Travel Time Index = 1.84


#####  Travel Time Index by Segments

In [195]:
seg = ['segment'+str(i) for i in range(1, 6)]
for s in seg:
    df_grids_seg = df_grids[df_grids[s] == 1]
    seg_tti = np.sum(df_grids_seg.DAILY_VHT) / np.sum(df_grids_seg.DAILY_VHT.values - df_grids_seg.DAILY_VHD.values)
    print(f'Daily Travel Time Index for {s} = {seg_tti:.2f}')

Daily Travel Time Index for segment1 = 1.63
Daily Travel Time Index for segment2 = 2.41
Daily Travel Time Index for segment3 = 2.49
Daily Travel Time Index for segment4 = 3.09
Daily Travel Time Index for segment5 = 1.87


#### Peak Congestion Index

Volume-to-Capacity Ratio weighted by VMT

In [196]:
def congestion_index(df, period='AM'):
    am_length, pm_length = 3, 4
    if period.upper() == 'AM':
        return np.dot(df.AM_VOL.values, df.AM_VMT.values) / np.dot((df.CAP_AB.values+df.CAP_BA.values)*am_length, df.AM_VMT.values)
    else:
        return np.dot(df.PM_VOL.values, df.PM_VMT.values) / np.dot((df.CAP_AB.values+df.CAP_BA.values)*pm_length, df.PM_VMT.values)

In [197]:
am_congestion_index = congestion_index(df_grids, period='AM')
pm_congestion_index = congestion_index(df_grids, period='PM')
print(f'AM Period Congestion Index (Weighted V/C) = {am_congestion_index:.2f}')
print(f'PM Period Congestion Index (Weighted V/C) = {pm_congestion_index:.2f}')

AM Period Congestion Index (Weighted V/C) = 0.36
PM Period Congestion Index (Weighted V/C) = 0.56


In [198]:
for s, p in itertools.product(seg, ('AM', 'PM')):
    df_grids_seg = df_grids[df_grids[s] == 1]
    ci = congestion_index(df_grids_seg, period=p)
    print(f'Period Congestion Index for {s} in {p} = {ci:.2f}')

Period Congestion Index for segment1 in AM = 0.36
Period Congestion Index for segment1 in PM = 0.57
Period Congestion Index for segment2 in AM = 0.48
Period Congestion Index for segment2 in PM = 0.82
Period Congestion Index for segment3 in AM = 0.34
Period Congestion Index for segment3 in PM = 0.48
Period Congestion Index for segment4 in AM = 0.33
Period Congestion Index for segment4 in PM = 0.45
Period Congestion Index for segment5 in AM = 0.46
Period Congestion Index for segment5 in PM = 0.70


#### Congestion Duration

In [199]:
# df_grids.CONG_TIME.describe()

In [200]:
def congestion_duration(df, trim_proportion=0.05):
    return trim_mean(df.CONG_TIME.values, trim_proportion)

In [201]:
cong_dur = congestion_duration(df_grids)
print(f'Congestion Duration (Minutes) = {cong_dur:.0f}')

Congestion Duration (Minutes) = 41


C:\Anaconda3\lib\site-packages\scipy\stats\stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


#### Average Commuting Time

In [220]:
skim_files = (
    r'STM\STM_A\03_Performance_Summarizer\04_Results\Zone_Skim_CAV_1_WK.csv',
    r'STM\STM_A\03_Performance_Summarizer\04_Results\Zone_Skim_CAV_2P_WK.csv',
    r'STM\STM_A\03_Performance_Summarizer\04_Results\Zone_Skim_REG_1_WK.csv',
    r'STM\STM_A\03_Performance_Summarizer\04_Results\Zone_Skim_REG_2P_WK.csv',
             )
skim_files = [os.path.join(workspace, scenario, f) for f in skim_files]
print("Input Skim Files:", skim_files)

Input Skim Files: ['L:\\DCS\\Projects\\_Legacy\\60563434_SWIFT\\400_Technical\\SWIFT_Workspace\\Scenarios\\Scenario_2017\\STM\\STM_A\\03_Performance_Summarizer\\04_Results\\Zone_Skim_CAV_1_WK.csv', 'L:\\DCS\\Projects\\_Legacy\\60563434_SWIFT\\400_Technical\\SWIFT_Workspace\\Scenarios\\Scenario_2017\\STM\\STM_A\\03_Performance_Summarizer\\04_Results\\Zone_Skim_CAV_2P_WK.csv', 'L:\\DCS\\Projects\\_Legacy\\60563434_SWIFT\\400_Technical\\SWIFT_Workspace\\Scenarios\\Scenario_2017\\STM\\STM_A\\03_Performance_Summarizer\\04_Results\\Zone_Skim_REG_1_WK.csv', 'L:\\DCS\\Projects\\_Legacy\\60563434_SWIFT\\400_Technical\\SWIFT_Workspace\\Scenarios\\Scenario_2017\\STM\\STM_A\\03_Performance_Summarizer\\04_Results\\Zone_Skim_REG_2P_WK.csv']


In [221]:
df_skim = pd.concat([pd.read_csv(f) for f in skim_files])

In [222]:
# df_skim.head()

In [223]:
def avg_commute_stats(df):
    avg_time = np.dot(df.TIME.values, df.COUNT.values) / np.sum(df.COUNT.values)
    avg_distance = np.dot(df.DISTANCE.values, df.COUNT.values) / np.sum(df.COUNT.values)
    avg_cost = np.dot(df.COST.values, df.COUNT.values) / np.sum(df.COUNT.values)
    
    return avg_time, avg_distance, avg_cost

In [224]:
avg_time, avg_dist, avg_cost = avg_commute_stats(df_skim)
print(f'Average Commute Time (minutes)   = {avg_time:.0f}')
print(f'Average Commute Distance (miles) = {avg_dist:.0f}')
print(f'Average Commute Cost (cents)     = {avg_cost:.0f}')

Average Commute Time (minutes)   = 19
Average Commute Distance (miles) = 11
Average Commute Cost (cents)     = 3
